# COMP 5212 Programming Assignment 1
## Image classification with Logistic Regression and Support Vector Machine

### Load data
* Load minist data and store the data to $home$ directory
* create dataloaders for training and testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#Don't change batch size
batch_size = 64

from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

In [3]:
train_data = datasets.MNIST('~/data/mnist', train=True, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                            ]))
test_data = datasets.MNIST('~/data/mnist', train=False, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ]))
subset_indices = ((train_data.train_labels == 0) +
                  (train_data.train_labels == 1)).nonzero().squeeze(1)
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=batch_size, shuffle=False, sampler=SubsetRandomSampler(subset_indices))


subset_indices = ((test_data.test_labels == 0) +
                  (test_data.test_labels == 1)).nonzero().squeeze(1)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=batch_size, shuffle=False, sampler=SubsetRandomSampler(subset_indices))


/home/dingdong/anaconda3/envs/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/dingdong/anaconda3/envs/venv/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


### Train model

In [4]:
from models import Logistic_Regression, logistic_loss, svm_loss
from torch import optim
import numpy as np

In [9]:
for i, (images, labels) in enumerate(train_loader):
    images = Variable(images.view(-1, 28*28))
    #Convert labels from 0,1 to -1,1
    labels = Variable(2*(labels.float()-0.5))
    break
images.shape, labels.shape
# Training the Model
# Notice that newest Pytorch merge tensor and variable, so the additional Variable wrapping is no longer required.

model = Logistic_Regression(28*28, 1)

# print(model(images))
loss_fn = svm_loss
print(loss_fn(model(images), labels))


tensor(24.3792, grad_fn=<DivBackward0>)


In [10]:
num_epochs = 20  # added by me. modifiable.
opt = optim.SGD(model.parameters(), lr=5e-3)
device = torch.device('cpu') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(num_epochs):
    total_loss = []
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28)).to(device)
        #Convert labels from 0,1 to -1,1
        labels = Variable(2*(labels.float()-0.5)).to(device)
        
        ## TODO 
        pred = model(images)
        loss = loss_fn(pred, labels)
        
        loss.backward()
        opt.step()
        opt.zero_grad()

        total_loss.append(loss.cpu().detach().numpy())
        # accuracy = 
    total_loss = np.mean(total_loss)
    #Print your results every epoch
    # with torch.no_grad():
    #     correct_test, total_test = 0, 0
    print(f"Epoch {epoch}, loss = {total_loss}")


Epoch 0, loss = 7.491518020629883
Epoch 1, loss = 0.7812595367431641
Epoch 2, loss = 0.40569907426834106
Epoch 3, loss = 0.2773037254810333
Epoch 4, loss = 0.21542297303676605
Epoch 5, loss = 0.1786784529685974
Epoch 6, loss = 0.1529337763786316
Epoch 7, loss = 0.13383746147155762
Epoch 8, loss = 0.12043789029121399
Epoch 9, loss = 0.11073549836874008
Epoch 10, loss = 0.10326942056417465
Epoch 11, loss = 0.09777548909187317
Epoch 12, loss = 0.09283657371997833
Epoch 13, loss = 0.0886225476861
Epoch 14, loss = 0.08512246608734131
Epoch 15, loss = 0.08229662477970123
Epoch 16, loss = 0.07913669943809509
Epoch 17, loss = 0.0762627124786377
Epoch 18, loss = 0.0736735388636589
Epoch 19, loss = 0.07133575528860092


### Test model

In [12]:
# Test the Model
correct = 0.
total = 0.
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28)).to(device)

    ## Put your prediction code here
    pred = model(images).squeeze()
    prediction = torch.sigmoid(pred).round().cpu().detach()

    correct += (prediction.view(-1).long() == labels).sum()
    total += images.shape[0]
print('Accuracy of the model on the test images: %f %%' % (100 * (correct.float() / total)))

Accuracy of the model on the test images: 99.338058 %
